In [3]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
import numpy as np
import pandas as pd
import random
import sys
import io
import re
from unidecode import unidecode

In [4]:
csv = 'brz_dictatorship.csv'
songs = pd.read_csv(csv)

In [7]:
def get_tokenized_lines(df):
    words = []
    
    for index, row in df['song_lyrics'].iteritems():
        row = str(row).lower()
        for line in row.split('|-|'):
            new_words = re.findall(r"\b[a-z']+\b", unidecode(line))
            words = words + new_words
        
    return words

In [8]:
all_lyric_lines = get_tokenized_lines(songs)

In [9]:
SEQ_LENGTH = 50 + 1
sequences = list()

for i in range(SEQ_LENGTH, len(all_lyric_lines)):
    seq = all_lyric_lines[i - SEQ_LENGTH: i]
    sequences.append(seq)

print('Total Sequences: %d' % len(sequences))

Total Sequences: 19402


In [10]:
def save_doc(lines, filename):
    for line in lines:
        data = ' '.join(line)
        
    '\n'.join(data)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [11]:
out_filename = 'sequences.txt'
save_doc(sequences, out_filename)

In [12]:
vocab = set(all_lyric_lines)

word_to_index = {w: i for i, w in enumerate(vocab)}
index_to_word = {i: w for w, i in word_to_index.items()}
word_indices = [word_to_index[word] for word in vocab]
vocab_size = len(vocab)

print('vocabulary size: {}'.format(vocab_size))

vocabulary size: 4074


In [13]:
def get_tokenized_lines(lines, seq_len):
    tokenized = np.zeros((len(lines), seq_len))
    
    for r, line in enumerate(lines):
        for c, word in enumerate(line):
            tokenized[r, c] = word_to_index[word]

    return tokenized

In [14]:
tokenized_seq = get_tokenized_lines(sequences, SEQ_LENGTH)

In [15]:
tokenized_seq[:, -1].shape

(19402,)

In [16]:
from keras.utils import to_categorical

X, y = tokenized_seq[:, :-1], tokenized_seq[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = len(X[0])

In [17]:
print("X_shape", X.shape)
print("y_shape", y.shape)

X_shape (19402, 50)
y_shape (19402, 4074)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, batch_size=128, epochs=50)

In [17]:
import matplotlib.pyplot as plt

In [18]:
plt.plot(history.history['loss'])

NameError: name 'history' is not defined

In [413]:
model.save('model.h5')

NameError: name 'dump' is not defined

In [19]:
from keras.models import load_model

model = load_model('model.h5')

In [21]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [34]:
# load cleaned text sequences
in_filename = 'data/clean_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = SEQ_LENGTH - 1

In [138]:
from random import randint

seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

and don't wanna diss niggas when write no more and don't wanna bring chicks to the crib to watch flick in the basement on the bed with dim lights no more now i'm losin feeling losin appealing need to step it up like prints from shoes on the ceiling oh damn



In [139]:
def texts_to_sequences(texts, word_to_index):
    indices = np.zeros((1, len(texts)), dtype=int)
    
    for i, text in enumerate(texts):
        indices[:, i] = word_to_index[text]
        
    return indices

In [140]:
def my_pad_sequences(seq, maxlen):
    start = seq.shape[1] - maxlen
    
    return seq[:, start: start + maxlen]

In [142]:
def generate_seq(model, word_to_index, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text

    for _ in range(n_words):
        encoded = texts_to_sequences(in_text.split()[1:], word_to_index)
        encoded = my_pad_sequences(encoded, maxlen=seq_length)
        
        yhat = model.predict_classes(encoded, verbose=0)
        out_word = ''
    
        for word, index in word_to_index.items():
            if index == yhat:
                out_word = word
                break
        
        in_text += ' ' + out_word
        result.append(out_word)
        
    return ' '.join(result)

In [143]:
generated = generate_seq(model, word_to_index, seq_length, seed_text, 50)
print(generated)

hahahaha i've banged calling lightskin matters cooking bill even it'd sweaters h overlook conversation banged pullin eatin recognize freshman six matters wolves resemble buddin mad growin sleepy cosign sum calling shut lie representing wits calling remixing sewed shrimp supported supported females luckily banged learn cater learn attempt representing matters rosetta
